# Cirq Basics

The primary representation of quantum programs in Cirq is the **Circuit** class. A Circuit is a collection of Moments. A **Moment** is a collection of Operations that all act during the same abstract time slice. An **Operation** is a some effect that operates on a specific subset of Qubits, the most common type of Operation is a GateOperation.

We can create a 3 by 3 grid of qubits using:

In [1]:
import cirq
qubits = [cirq.GridQubit(x, y) for x in range(3) for y in range(3)]
for i in range(len(qubits)): print(qubits[i])

(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
(2, 0)
(2, 1)
(2, 2)


A Gate represents a physical process that occurs on a Qubit. The important property of a Gate is that it can be applied on to one or more qubits. This can be done via the Gate.on method itself or via () and doing this turns the Gate into a GateOperation.

Apply Pauli X Gate to the qubit at location (0, 0)

In [7]:
x_gate = cirq.X
x_op = x_gate(qubits[4])
print(x_op)

X((1, 1))


A **Moment** is simply a collection of operations, each of which operates on a different set of qubits, and which conceptually represents these operations as occurring during this abstract time slice. 
For example, here is a Moment in which Pauli X and a CNOT gate operate on three qubits:

In [14]:
cnot = cirq.CNOT(qubits[0], qubits[1])
x = cirq.X(qubits[2])
moment = cirq.Moment([x, cnot])
print(moment)

X((0, 2)) and CNOT((0, 0), (0, 1))


Finally, a **Circuit** is an ordered series of Moments. 
For example, here is a simple circuit made up of two moments:

In [17]:
cz01 = cirq.CZ(qubits[0], qubits[1])
x2 = cirq.X(qubits[2])
cz12 = cirq.CZ(qubits[1], qubits[2])
moment0 = cirq.Moment([cz01, x2])
moment1 = cirq.Moment([cz12])
circuit = cirq.Circuit((moment0, moment1))
print(circuit)

(0, 0): ───@───────
           │
(0, 1): ───@───@───
               │
(0, 2): ───X───@───


The above constructions are purely to illustrate the conceptual ideas behind building blocks of Cirq.

**Circuit.append** method can be used to construct circuits more efficiently:

In [21]:
from cirq.ops import CZ, H, CNOT
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1), H(q2), H(q0), CZ(q1, q2), CNOT(q1, q2)])
print(circuit)

(0, 0): ───@───H───────
           │
(1, 0): ───@───@───@───
               │   │
(2, 0): ───H───@───X───


**InsertStrategy** defines how Operations are placed in a Circuit when requested to be inserted at a given location or index of Moment. 
There are four such strategies: 
InsertStrategy.EARLIEST, InsertStrategy.NEW, InsertStrategy.INLINE and InsertStrategy.NEW_THEN_INLINE.

In [30]:
from cirq.circuits import InsertStrategy
circuit = cirq.Circuit()
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.NEW)
print(circuit)

(0, 0): ───H───────────

(1, 0): ───────H───────

(2, 0): ───────────H───


The default default - InsertStrategy.NEW_THEN_INLINE: Creates a new moment at the desired insert location for the first operation, but then switches to inserting operations according to InsertStrategy.INLINE.

Finally, Circuits can be iterated over and sliced. When they are iterated over each item in the iteration is a moment.